<a href="https://colab.research.google.com/github/yoseforaz0990/ML-templates/blob/main/deep_learning/ResUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Step Name                                | Description                                                                                                             |
|------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|
| Importing necessary libraries           | Importing the required libraries for building the ResUNet model.                                                        |
| Defining the Residual Block             | Defining a single residual block with skip connection.                                                                  |
| Creating the ResUNet model              | Building the ResUNet model using the defined residual blocks, encoder, bridge, and decoder.                            |
| Compiling the ResUNet model             | Compiling the ResUNet model with binary cross-entropy loss and Adam optimizer.                                          |
| Printing the ResUNet model summary      | Displaying the summary of the ResUNet model architecture.                                                               |


In [ ]:
# Step 1: Importing necessary libraries
# Description: Importing the required libraries for building the ResUNet model.

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, UpSampling2D, concatenate, Add
from tensorflow.keras.models import Model

# Step 2: Defining the Residual Block
# Description: Defining a single residual block with skip connection.

def residual_block(x, filters, kernel_size=(3, 3), activation="relu"):
    identity = x

    x = Conv2D(filters, kernel_size, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)

    x = Conv2D(filters, kernel_size, padding="same")(x)
    x = BatchNormalization()(x)

    if x.shape[-1] != filters:
        identity = Conv2D(filters, (1, 1), padding="same")(identity)
        identity = BatchNormalization()(identity)

    x = Add()([x, identity])
    x = Activation(activation)(x)
    return x

# Step 3: Creating the ResUNet model
# Description: Building the ResUNet model using the defined residual blocks.

def create_resunet(input_shape=(256, 256, 3), num_classes=1, num_filters=64):
    inputs = Input(shape=input_shape)

    # Encoder
    x = Conv2D(num_filters, (3, 3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = residual_block(x, num_filters)
    x = MaxPooling2D((2, 2))(x)

    num_filters *= 2
    x = residual_block(x, num_filters)
    x = MaxPooling2D((2, 2))(x)

    num_filters *= 2
    x = residual_block(x, num_filters)
    x = MaxPooling2D((2, 2))(x)

    # Bridge
    x = residual_block(x, num_filters)

    # Decoder
    num_filters //= 2
    x = UpSampling2D((2, 2))(x)
    x = residual_block(x, num_filters)

    num_filters //= 2
    x = UpSampling2D((2, 2))(x)
    x = residual_block(x, num_filters)

    num_filters //= 2
    x = UpSampling2D((2, 2))(x)
    x = residual_block(x, num_filters)

    outputs = Conv2D(num_classes, (1, 1), activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Step 4: Compiling the ResUNet model
# Description: Compiling the ResUNet model with binary cross-entropy loss and Adam optimizer.

model = create_resunet()
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Step 5: Printing the ResUNet model summary
# Description: Displaying the summary of the ResUNet model architecture.

model.summary()
